In [17]:
# coding: utf-8

import warnings
warnings.filterwarnings('ignore')

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

import json
import pickle
import re
import nltk
from urllib.parse import unquote,urlparse
import pandas as pd
from keras.preprocessing.sequence import pad_sequences
import numpy
from kafka import KafkaConsumer

with open("SVM_model","rb")as f:
    model=pickle.load(f)

    
with open("word2vec.pickle","rb") as f :
    word2vec=pickle.load(f)
    dictionary=word2vec["dictionary"]
    embeddings=word2vec["embeddings"]
    reverse_dictionary = word2vec["reverse_dictionary"]


def to_index(data):
    d_index=[]
    for word in data:
        if word in dictionary.keys():
            d_index.append(dictionary[word])
        else:
            d_index.append(dictionary["UNK"])
    return d_index


def GeneSeg(payload):
    #数字泛化为"0"
    payload=payload.lower()
    payload=unquote(unquote(payload))
    payload,num=re.subn(r'\d+',"0",payload)
    #替换url为”http://u
    payload,num=re.subn(r'(http|https)://[a-zA-Z0-9\.@&/#!#\?]+', "http://u", payload)
    #分词
    r = '''
        (?x)[\w\.]+?\(
        |\)
        |"\w+?"
        |'\w+?'
        |http://\w
        |</\w+>
        |<\w+>
        |<\w+
        |\w+=
        |>
        |[\w\.]+
    '''

    return nltk.regexp_tokenize(payload, r)

def xss_detection(payload):
    xssed_data=[]
    maxlen=200
    word=GeneSeg(payload)
    xssed_data.append(word)
    #print(xssed_data)
    data=xssed_data[0]
    #print(type(data))
    datas_index=[to_index(data)]
    datas_index=pad_sequences(datas_index,value=-1,maxlen=maxlen)
    datas_embed=[]
    dims=len(embeddings["UNK"])
    for datas_index in datas_index:
        data_embed=[]
        for d in datas_index:
            if d != -1:
                data_embed.extend(embeddings[reverse_dictionary[d]])
            else:
                data_embed.extend([0.0] * dims)
        datas_embed.append(data_embed)

    data=(datas_embed[0])
    result=model.predict([data])
    if result==numpy.array([1]):
        return True
    else:
        return False

    
def xss(url):
    parsed_url=urlparse(url)
    query=parsed_url.query
    #print(query)
    if query=='':
        return 0
    else:
        try:
            variables = query.split('&')
            values = [ variable.split('=')[1] for variable in variables if len(variable.split('='))>1 ]
            for value in values:
                result=xss_detection(value)
                if result==True:
                    return 1 #print("may be xss_attack:",url)
                else:
                    return 0
        except Exception as err:
            return 0
            #pass
            #print(str(err))

    #if linecount%1000==0:
        #print("processed:",linecount)


xss('http://br-ofertasimperdiveis.epizy.com/produto.php?linkcompleto=iphone-6-plus-apple-64gb-cinza')
xss('http://my-softbank-security.com/wap_login.htm')





0

In [18]:
consumer = KafkaConsumer('http')
for message in consumer:
    print(message)
    # message value and key are raw bytes -- decode if necessary!
    # e.g., for unicode: `message.value.decode('utf-8')`
    http_log = json.loads(message.value.decode('utf-8'))
    http_query = http_log["host"]+http_log["uri"]
    #print(http_query)
    if xss(http_query):
        print(http_query," XSS ")
    """
    dns_query = http_log["query"]
    x = domain2list(extract(dns_query).domain)
    X = mat([x,])
    rs = results[model.predict_classes(X)[0][0]]
    
    print(rs, dns_query)
    """
    print(http_query)

ConsumerRecord(topic='http', partition=0, offset=688, timestamp=-1, timestamp_type=0, key=None, value=b'{"ts":1538209508.209421,"uid":"Cwufom1LRTgBSsNmv7","id.orig_h":"192.168.1.190","id.orig_p":48590,"id.resp_h":"220.181.112.244","id.resp_p":80,"trans_depth":1,"method":"GET","host":"www.baidu.com","uri":"/index.html?aaa=<script>alert(1)</script>","user_agent":"curl/7.59.0","request_body_len":0,"response_body_len":0,"tags":[]}', checksum=-929890586, serialized_key_size=-1, serialized_value_size=326)
www.baidu.com/index.html?aaa=<script>alert(1)</script>  XSS 
www.baidu.com/index.html?aaa=<script>alert(1)</script>
ConsumerRecord(topic='http', partition=0, offset=689, timestamp=-1, timestamp_type=0, key=None, value=b'{"ts":1538209513.931372,"uid":"CNqZfH2Q9FYXd5fs33","id.orig_h":"192.168.1.190","id.orig_p":45492,"id.resp_h":"140.211.169.196","id.resp_p":80,"trans_depth":1,"method":"GET","host":"fedoraproject.org","uri":"/static/hotspot.txt","version":"1.1","request_body_len":0,"response_

KeyboardInterrupt: 